# Mortgage Workflow

## The Dataset
The dataset used with this workflow is derived from [Fannie Mae’s Single-Family Loan Performance Data](http://www.fanniemae.com/portal/funding-the-market/data/loan-performance-data.html) with all rights reserved by Fannie Mae. This processed dataset is redistributed with permission and consent from Fannie Mae.

To acquire this dataset, please visit [RAPIDS Datasets Homepage](https://docs.rapids.ai/datasets/mortgage-data)

## Introduction
The Mortgage workflow is composed of three core phases:

1. ETL - Extract, Transform, Load
2. Data Conversion
3. ML - Training

### ETL
Data is 
1. Read in from storage
2. Transformed to emphasize key features
3. Loaded into volatile memory for conversion

### Data Conversion
Features are
1. Broken into (labels, data) pairs
2. Distributed across many workers
3. Converted into compressed sparse row (CSR) matrix format for XGBoost

### Machine Learning
The CSR data is fed into a distributed training session with Dask-XGBoost

In [1]:
import numpy as np
import dask
import dask_cudf
from dask_cuda import LocalCUDACluster
from dask.delayed import delayed
from dask.distributed import Client, wait
from collections import OrderedDict
from glob import glob
import os

In [2]:
#Create the cluster
import subprocess

cmd = "hostname --all-ip-addresses"
process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
output, error = process.communicate()
IPADDR = str(output.decode()).split()[0]

cluster = LocalCUDACluster(ip=IPADDR)
client = Client(cluster)
client

Client Scheduler: tcp://10.136.7.103:42260 Dashboard: http://10.136.7.103:8787/status,Cluster Workers: 4 Cores: 4 Memory: 404.34 GB


In [3]:
import cudf
#Specify Path to the files
acq_data_path = "/data/kratos/mortgage_1yr/acq/*"
perf_data_path = "/data/kratos/mortgage_1yr/perf/*"
col_names_path = "/data/kratos/mortgage_1yr/names.csv"

chunksize = "500MB" #Chunksize for reading the data

In [4]:
# Loading the three files
acq_df_cols = OrderedDict([
        ("loan_id", "int64"),
        ("orig_channel", "category"),
        ("seller_name", "str"),
        ("orig_interest_rate", "float64"),
        ("orig_upb", "int64"),
        ("orig_loan_term", "int64"),
        ("orig_date", "date"),
        ("first_pay_date", "date"),
        ("orig_ltv", "float64"),
        ("orig_cltv", "float64"),
        ("num_borrowers", "float64"),
        ("dti", "float64"),
        ("borrower_credit_score", "float64"),
        ("first_home_buyer", "category"),
        ("loan_purpose", "category"),
        ("property_type", "category"),
        ("num_units", "int64"),
        ("occupancy_status", "category"),
        ("property_state", "category"),
        ("zip", "int64"),
        ("mortgage_insurance_percent", "float64"),
        ("product_type", "category"),
        ("coborrow_credit_score", "float64"),
        ("mortgage_insurance_type", "float64"),
        ("relocation_mortgage_indicator", "category")
    ])
    
acq_df = dask_cudf.read_csv(acq_data_path, names=list(acq_df_cols.keys()), delimiter='|', dtype=list(acq_df_cols.values()), chunksize=chunksize)


perf_df_cols = OrderedDict([
    ("loan_id", "int64"),
    ("monthly_reporting_period", "date"),
    ("servicer", "category"),
    ("interest_rate", "float64"),
    ("current_actual_upb", "float64"),
    ("loan_age", "float64"),
    ("remaining_months_to_legal_maturity", "float64"),
    ("adj_remaining_months_to_maturity", "float64"),
    ("maturity_date", "date"),
    ("msa", "float64"),
    ("current_loan_delinquency_status", "int32"),
    ("mod_flag", "category"),
    ("zero_balance_code", "category"),
    ("zero_balance_effective_date", "date"),
    ("last_paid_installment_date", "date"),
    ("foreclosed_after", "date"),
    ("disposition_date", "date"),
    ("foreclosure_costs", "float64"),
    ("prop_preservation_and_repair_costs", "float64"),
    ("asset_recovery_costs", "float64"),
    ("misc_holding_expenses", "float64"),
    ("holding_taxes", "float64"),
    ("net_sale_proceeds", "float64"),
    ("credit_enhancement_proceeds", "float64"),
    ("repurchase_make_whole_proceeds", "float64"),
    ("other_foreclosure_proceeds", "float64"),
    ("non_interest_bearing_upb", "float64"),
    ("principal_forgiveness_upb", "float64"),
    ("repurchase_make_whole_proceeds_flag", "category"),
    ("foreclosure_principal_write_off_amount", "float64"),
    ("servicing_activity_indicator", "category")
])
perf_df = dask_cudf.read_csv(perf_data_path, names=list(perf_df_cols.keys()), delimiter='|', dtype=list(perf_df_cols.values()), chunksize=chunksize)


names_df_cols = OrderedDict([
        ("seller_name", "str"),
        ("new", "str"),
])
names_df = dask_cudf.read_csv("file://"+col_names_path, names=list(names_df_cols.keys()), delimiter='|', dtype=list(names_df_cols.values()),skiprows=1)

Let us look at the seller_names from the acquisisiton df

Same sellers sometimes have different names in the dataset:
```
JP MORGAN CHASE BANK, NA
JPMORGAN CHASE BANK, NA
JPMORGAN CHASE BANK, NATIONAL ASSOCIATION
```
To handle this we have a `names.csv` files that has mappings of seller_names to a common seller_name

In [5]:
#TODO names.csv mapping to df

Boolean feature indicating if a load has ever been delinquent over 30 days, 90 days and 180 days

In [6]:
delinq_status_df = perf_df[['loan_id','current_loan_delinquency_status']].groupby("loan_id").current_loan_delinquency_status.max().to_frame()
delinq_status_df['delinq30'] = (delinq_status_df['current_loan_delinquency_status'] >= 1)
delinq_status_df['delinq90'] = (delinq_status_df['current_loan_delinquency_status'] >= 3)
delinq_status_df['delinq180'] = (delinq_status_df['current_loan_delinquency_status'] >= 6)

Feature indicating first time when loan became delinquent over 30, 90 and 180 days

In [7]:
temp = perf_df[['current_loan_delinquency_status',
                'loan_id',
                'monthly_reporting_period']].query("current_loan_delinquency_status >= 1").groupby("loan_id").monthly_reporting_period.min().to_frame()
temp['delinquency_30'] = temp['monthly_reporting_period']
temp = temp.map_partitions(cudf.DataFrame.drop,'monthly_reporting_period')
temp = temp.map_partitions(cudf.DataFrame.reset_index)
temp2 = perf_df[['current_loan_delinquency_status',
                'loan_id',
                'monthly_reporting_period']].query("current_loan_delinquency_status >= 3").groupby("loan_id").monthly_reporting_period.min().to_frame()
temp2['delinquency_90'] = temp2['monthly_reporting_period']
temp2 = temp2.map_partitions(cudf.DataFrame.drop,'monthly_reporting_period')
temp2 = temp2.map_partitions(cudf.DataFrame.reset_index)
temp3 = perf_df[['current_loan_delinquency_status',
                'loan_id',
                'monthly_reporting_period']].query("current_loan_delinquency_status >= 6").groupby("loan_id").monthly_reporting_period.min().to_frame()
temp3['delinquency_180'] = temp3['monthly_reporting_period']
temp3 = temp3.map_partitions(cudf.DataFrame.drop,'monthly_reporting_period')
temp3 = temp3.map_partitions(cudf.DataFrame.reset_index)
temp = temp.merge(temp2, on = "loan_id", how = "left")
temp = temp.merge(temp3, on = "loan_id", how = "left")

Over a rolling_window of width 1 year and stride one month find a feature which has 3 values 0,1,2 and is the sum of the booleans is the loan delinquent over 30 days in the 1 year period and is the current upb 0

In [ ]:
perf_df.head()

In [ ]:
perf_df_tt = perf_df[['loan_id','loan_age']].rolling(2).mean()
perf_df_tt.head()

#### Approach 1

In [8]:
delinq_status_df2 = delinq_status_df.map_partitions(cudf.DataFrame.reset_index)
delinq_status_df2 = delinq_status_df2.map_partitions(cudf.DataFrame.drop,"current_loan_delinquency_status")
perf_df_1 = perf_df.merge(delinq_status_df2, on = "loan_id",how="left")

In [9]:
perf_df_1 = perf_df_1.merge(temp, on = "loan_id", how = "left")

In [11]:
%time len(perf_df_1)

CPU times: user 476 ms, sys: 28 ms, total: 504 ms
Wall time: 3.01 s


36190382

#### Approach 2

In [12]:
delinq_status_df3 = delinq_status_df.map_partitions(cudf.DataFrame.reset_index)
delinq_status_df3 = delinq_status_df3.map_partitions(cudf.DataFrame.drop,"current_loan_delinquency_status")
delinq_status_df3 = delinq_status_df3.merge(temp, on = "loan_id", how = "left")

In [13]:
perf_df_2 = perf_df.merge(delinq_status_df3, on = "loan_id", how = "left")

In [14]:
%time len(perf_df_2)

CPU times: user 508 ms, sys: 40 ms, total: 548 ms
Wall time: 3.11 s


36190382

Create rolling feature

In [ ]:
#Set Index approach
perf_df = perf_df.set_index("loan_id")